# 문맥 단어 활용

먼저 간단한 US_ZIP_CODE인식기를 만들어 보겠습니다.

In [1]:
from presidio_analyzer import (
    Pattern,
    PatternRecognizer,
    RecognizerRegistry,
    AnalyzerEngine,
)

# Define the regex pattern
regex = r"(\b\d{5}(?:\-\d{4})?\b)"  # very weak regex pattern
zipcode_pattern = Pattern(name="zip code (weak)", regex=regex, score=0.01)

# Define the recognizer with the defined pattern
zipcode_recognizer = PatternRecognizer(
    supported_entity="US_ZIP_CODE", patterns=[zipcode_pattern]
)

registry = RecognizerRegistry()
registry.add_recognizer(zipcode_recognizer)
analyzer = AnalyzerEngine(registry=registry)

# Test
results = analyzer.analyze(text="My zip code is 90210", language="en")

print(f"Result:\n {results}")


Result:
 [type: US_ZIP_CODE, start: 15, end: 20, score: 0.01]


이것은 작동하지만 5자리 문자열을 잡을 것입니다. 이것이 우리가 점수를 0.01로 설정한 이유입니다. 점수를 높이기 위해 문맥 단어를 사용합시다:

In [3]:
from presidio_analyzer import PatternRecognizer

# Define the recognizer with the defined pattern and context words
zipcode_recognizer_w_context = PatternRecognizer(
    supported_entity = "US_ZIP_CODE",
    patterns=[zipcode_pattern],
    context=["zip", "zipcode"],
)

In [4]:
from presidio_analyzer import AnalyzerEngine, RecognizerRegistry

registry = RecognizerRegistry()
registry.add_recognizer(zipcode_recognizer_w_context)
analyzer = AnalyzerEngine(registry=registry)

# Test
results = analyzer.analyze(text="My zip code is 90210", language="en")
print("Result:")
print(results)

Result:
[type: US_ZIP_CODE, start: 15, end: 20, score: 0.4]


생성할 때 매개변수 AnalyzerEngine에 전달하여 자체 컨텍스트 향상 논리를 제공할 수 있습니다 context_aware_enhancer. 전달되지 않으면 기본적으로 AnalyzerEngine생성 되며, 인식기가 컨텍스트 단어를 보유하고 해당 단어의 보조 정리가 일치하는 엔터티 주변에서 발견되는 경우 일치하는 각 결과의 점수를 향상시킵니다.LemmaContextAwareEnhancer

# AnalyzerEngine새 인식기 생성 및 추가:

In [5]:
from presidio_analyzer import AnalyzerEngine, RecognizerRegistry

registry = RecognizerRegistry()
registry.add_recognizer(zipcode_recognizer_w_context)
analyzer = AnalyzerEngine(registry=registry)

# Test
results = analyzer.analyze(text="My zip code is 90210", language="en")
print("Result:")
print(results)

Result:
[type: US_ZIP_CODE, start: 15, end: 20, score: 0.4]


LemmaContextAwareEnhancer기본 컨텍스트 유사성 계수가 0.35이고 컨텍스트 유사성이 있는 기본 최소 점수가 0.4 이기 때문에 신뢰 점수는 이제 0.01이 아닌 0.4입니다. 객체 의 context_similarity_factor및 min_score_with_context_similarity매개변수에 다른 값을 전달하여 변경할 수 있습니다 . 

LemmaContextAwareEnhancer예를 들어:

In [8]:
from presidio_analyzer import AnalyzerEngine, RecognizerRegistry
from presidio_analyzer.context_aware_enhancers import LemmaContextAwareEnhancer

context_aware_enhancer = LemmaContextAwareEnhancer(
    context_similarity_factor=0.45, min_score_with_context_similarity=0.4
)

registry = RecognizerRegistry()
registry.add_recognizer(zipcode_recognizer)
analyzer = AnalyzerEngine(
    registry=registry, context_aware_enhancer=context_aware_enhancer
)

# Test
results = analyzer.analyze(text="My zip code is 90210", language="en")
print("Result:")
print(results)

Result:
[type: US_ZIP_CODE, start: 15, end: 20, score: 0.01]


신뢰도 점수는 0.01에서 0.45로 향상되었으며 최소값인 0.4보다 높기 때문에 이제 0.46입니다.

주변 단어 외에도 추가 컨텍스트 단어가 요청 수준에서 전달될 수 있습니다. 이것은 열 이름 또는 특정 사용자 입력과 같은 메타데이터에서 오는 컨텍스트가 있을 때 유용합니다. 다음 예에서 "zip" 컨텍스트 단어가 텍스트에 나타나지 않지만 여전히 신뢰도 점수를 0.01에서 0.4로 향상시키는 방법에 주목하십시오.

In [10]:
from presidio_analyzer import AnalyzerEngine, RecognizerRegistry, PatternRecognizer

# Define the recognizer with the defined pattern and context words
zipcode_recognizer = PatternRecognizer(
    supported_entity="US_ZIP_CODE",
    patterns=[zipcode_pattern],
    context=["zip", "zipcode"],
)
registry = RecognizerRegistry()
registry.add_recognizer(zipcode_recognizer)
analyzer = AnalyzerEngine(registry=registry)

# Test with an example record having a column name which could be injected as context
record = {"column_name": "zip", "text": "My code is 90210"}

result = analyzer.analyze(
    text=record["text"], language="en", context=[record["column_name"]]
)

print("Result:")
print(result)


Result:
[type: US_ZIP_CODE, start: 11, end: 16, score: 0.4]


# 예 7: 결정 프로세스 추적

우편 번호 값이 감지된 방법을 이해하기 위해 결정 프로세스 출력을 사용하겠습니다.

In [1]:
from presidio_analyzer import AnalyzerEngine
import pprint

analyzer = AnalyzerEngine()

results = analyzer.analyze(
    text="My zip code is 90210", language="en", return_decision_process=True
)

decision_process = results[0].analysis_explanation

pp = pprint.PrettyPrinter()
print("Decision process output:\n")
pp.pprint(decision_process.__dict__)


Decision process output:

{'original_score': 0.85,
 'pattern': None,
 'pattern_name': None,
 'recognizer': 'SpacyRecognizer',
 'score': 0.85,
 'score_context_improvement': 0,
 'supportive_context_word': '',
 'textual_explanation': "Identified as DATE by Spacy's Named Entity "
                        'Recognition',
 'validation_result': None}
